
# Location Web Scraping of Microsoft Academics Graph (MAG) Dataset

Jupyter Notebook for the web scraping of the conferences locations of the Microsoft Academics Graph (MAG) dump.

For this process, the following CSV file is needed: ```out_mag_citations_count_and_conferences.csv```. 
The above file must be generated running the ```preprocess_mag.ipynb``` Notebook that is contained in the ```1 - Citation Dumps Preprocess``` folder of this Repository.

In particular, the following operations are going to be executed:
* Opening of the CSV peprocessed dump
* Fix of the conferences names
* Obtaining the missing locations with queries to the DBLP website
* Fix of the locations format

Lastly, the entire preprocessed dump is going to be saved on disk in CSV format

In [ ]:
# Libraries Import
import pandas as pd
import platform
import multiprocessing as mp 
import concurrent       
from preprocess_multithread_utils import * 

pd.set_option('display.max_columns', None)

## File Paths
Please set your working directory paths.

In [ ]:
# ******************* PATHS ********************+

# Dumps Directory Path
path_file_import = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Import/'

# CSV Exports Directory Path
path_file_export = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/'

### Multithreading Settings
Settings needed for the multithreaded queries to gather the missing conferences locations from the DBLP website.

Please specify the number of CPU threads below:

In [ ]:
n_cpu_threads = 8 # Number of CPU threads

Special setting for the specific operating systems.

**Note**: Due to the latest MacOS releases' security measures, we need to use the spawn method instead of fork.

In [ ]:
print(f"Notebook running on {platform.system()} OS: ")

if platform.system() == "Darwin" or platform.system() == "Windows": # MacOS and windows
    mp_ctx = mp.get_context("spawn")
    print("Spawn method has been set")
    
else: # other unix systems
    mp_ctx = mp.get_context("fork")
    print("Spawn method has been set")

## Read of the CSV Preprocessed Dump

In [ ]:
df_mag_preprocessed = pd.read_csv(path_file_export + 'out_mag_citations_count_and_conferences.csv', low_memory=False)
df_mag_preprocessed

## Fix of the Missing Conferences Names
Some papers have only the indication of the conference series. For this reason, the conference instance and the related conference locations don't have a value.

However, every paper has been published in a specific "instance" of a conference, hence it should have a location. These papers will be "fixed" considering the year of their publication and their conference.

In [ ]:
df_mag_preprocessed_subset = df_mag_preprocessed.iloc[:50]
df_mag_preprocessed_subset = df_mag_preprocessed_subset.dropna(subset = ['ConferenceNormalizedName'])
df_mag_preprocessed_subset.iloc[:10][["Year", "ConferenceSeriesNormalizedName", "ConferenceNormalizedName", "ConferenceDisplayName"]]

As you can see in the above test, the ConferenceNormalizedName seems to be made by the concatenation of ConferenceSeriesNormalizedName in lowercase, a space, and the papers' year.

**Note**: in the above subset the ConferenceDisplayName seems to be composed in the same way of ConferenceNormalizedName, but without the lowercase. However, this is not always true!

Now we're going to populate the ConferenceNormalizedName instances that don't have a value.

In [ ]:
df_mag_preprocessed.ConferenceNormalizedName.fillna(df_mag_preprocessed.ConferenceSeriesNormalizedName.str.lower() + ' ' + df_mag_preprocessed.Year.astype(str), inplace=True)
df_mag_preprocessed.iloc[:5]

I tried to do a new merge with the Conference Instances dataframe (this time it will be made on the ConferenceNormalizedName column), but I had no luck: these conference instances are missing. That's probably the reason of the NaN values in the ConferenceInstanceID field of the original Papers table.

In [ ]:
df_mag_conf_instances.loc[df_mag_conf_instances["ConferenceNormalizedName"] == "enter 2013"]

## Obtaining the Missing Conferences Locations from the DBLP Website
The missing conferences locations are going to be obtained from queries to the DBLP Website.

In [ ]:
df_mag_conferences = df_mag_preprocessed[["ConferenceNormalizedName", "ConferenceLocation"]]

In [ ]:
## TODO TEST
df_mag_conferences = pd.read_csv(path_file_export + 'out_mag_tmp_locations_subset.csv', low_memory=False)

Drop of the papers that don't need their location to be fixed.

In [ ]:
df_mag_conferences = df_mag_conferences[df_mag_conferences["ConferenceLocation"].isna()]
df_mag_conferences

Drop of the duplicated conferences. We only need unique values.

In [ ]:
df_mag_conferences = df_mag_conferences.drop_duplicates(subset="ConferenceNormalizedName")

print(f"Now we only need to search for the location of {df_mag_conferences.__len__()} unique conferences")

### Define of the Web Scraping Function
We'll do a web scraping in two different URL formats, hence the need of two web scraping phases (with two different functions that are going to be passed as parameter).

In [ ]:
def dblp_location_scraper(conferences_dataframe, mt_downloader_operation_function, dblp_url = "https://dblp.org/db/conf/"):
    dict_conf_locations = {}      
    download_list = list(conferences_dataframe.ConferenceNormalizedName.values)

    executor = concurrent.futures.ProcessPoolExecutor(max_workers=n_cpu_threads * 10, mp_context=mp_ctx)
    futures = [executor.submit(mt_downloader_operation_function, conf_name, dblp_url) for conf_name in download_list]

    for future in concurrent.futures.as_completed(futures):
        try:
            k, v = future.result()
        except Exception as e:
            print(f"{futures[future]} throws {e}")
        else:
            dict_conf_locations[k] = v
            pass

    # Converting the resulting dictionary to a dataframe
    df_conf_locations = pd.DataFrame(dict_conf_locations.items(), columns=['ConferenceNormalizedName', 'ConferenceLocation'])

    return df_conf_locations

### Web Scraping Phase 1

#### Queries to https://dblp.org/db/conf/CONF_NAME/CONF_NAMEYEAR.html

Parallel execution of the queries to the DBLP website.

**Note**: this operation should take less than 20min, depending on your Internet speed.

In [ ]:
# TODO TEST
df_mag_conferences_v1_1_subset = df_mag_conferences.iloc[0:1000]
df_conf_locations_v1_1 = dblp_location_scraper(df_mag_conferences_v1_1_subset, mt_get_mag_conf_location_from_dblp_operation_v1, "https://dblp.org/db/conf/")

In [ ]:
df_conf_locations_v1_1 = dblp_location_scraper(df_mag_conferences, mt_get_mag_conf_location_from_dblp_operation_v1, "https://dblp.org/db/conf/")

Let's see how many conference locations have been fixed.

In [ ]:
df_conf_locations_v1_1 = df_conf_locations_v1_1.dropna(subset = ['ConferenceLocation'])

print(f"Fixed {len(df_conf_locations_v1_1.index)} over {len(df_mag_conferences.index)} unique conferences")

#### Queries to https://dblp.org/db/series/CONF_NAME/CONF_NAMEYEAR.html

We're going to try to get more location composing the URL in a different way.

**Note**: this operation should take less than 20min, depending on your Internet speed.

**Note**: in my tests, this method gave no results. I decided to leave the original code, in case something will change on the DBLP website. You can execute the download anyway if you want, by editing the following value. 

In [ ]:
download_anyway = False

First of all, we have to filter the conferences that have already been obtained:

In [ ]:
rows_to_drop = df_mag_conferences["ConferenceNormalizedName"].isin(df_conf_locations_v1_1["ConferenceNormalizedName"])
df_mag_conferences.drop(df_mag_conferences[rows_to_drop].index, inplace=True)

print(f"Now we only need to search for the location of {df_mag_conferences.__len__()} unique conferences")

In [ ]:
if download_anyway:
    df_conf_locations_v1_2 = dblp_location_scraper(df_mag_conferences, mt_get_mag_conf_location_from_dblp_operation_v1, "https://dblp.org/db/series/")

Let's see how many conference locations have been fixed.

In [ ]:
if download_anyway:
    df_conf_locations_v1_2 = df_conf_locations_v1_2.dropna(subset = ['ConferenceLocation'])

    print(f"Fixed {len(df_conf_locations_v1_2.index)} over {len(df_mag_conferences.index)} unique conferences")

### Web Scraping Phase 2

#### Queries to https://dblp.org/db/conf/CONF_NAME/index.html

Parallel execution of the queries to the DBLP website.

**Note**: this operation should take less than 20min, depending on your Internet speed.

First of all, we have to filter the conferences that have already been obtained:

In [ ]:
if download_anyway:
    rows_to_drop = df_mag_conferences["ConferenceNormalizedName"].isin(df_conf_locations_v1_2["ConferenceNormalizedName"])
    df_mag_conferences.drop(df_mag_conferences[rows_to_drop].index, inplace=True)

print(f"Now we only need to search for the location of {df_mag_conferences.__len__()} unique conferences")

In [ ]:
# TODO TEST
df_mag_conferences_v2_1_subset = df_mag_conferences.iloc[0:10]
df_mag_locations_v2_1 = dblp_location_scraper(df_mag_conferences_v2_1_subset, mt_get_mag_conf_location_from_dblp_operation_v2, "https://dblp.org/db/conf/")

In [ ]:
df_mag_locations_v2_1 = dblp_location_scraper(df_mag_conferences, mt_get_mag_conf_location_from_dblp_operation_v2, "https://dblp.org/db/conf/")

#### Queries to https://dblp.org/db/series/CONF_NAME/index.html

Parallel execution of the queries to the DBLP website.

**Note**: this operation should take less than 20min, depending on your Internet speed.

First of all, we have to filter the conferences that have already been obtained:

In [ ]:
rows_to_drop = df_mag_conferences["ConferenceNormalizedName"].isin(df_conf_locations_v2_1["ConferenceNormalizedName"])
df_mag_conferences.drop(df_mag_conferences[rows_to_drop].index, inplace=True)

print(f"Now we only need to search for the location of {df_mag_conferences.__len__()} unique conferences")

In [ ]:
df_mag_locations_v2_2 = dblp_location_scraper(df_mag_conferences, mt_get_mag_conf_location_from_dblp_operation_v2, "https://dblp.org/db/series/")

## Join of the New Location Data with the Original Dataframe

In [ ]:
# Merge with the first location dataframe
df_mag_preprocessed = pd.merge(df_mag_preprocessed, df_conf_locations_v1_1, on=['ConferenceNormalizedName'], how='left')

# Combine the two columns
df_mag_preprocessed['ConferenceLocation_x'] = df_mag_preprocessed['ConferenceLocation_x'].fillna(df_mag_preprocessed['ConferenceLocation_y'])
df_mag_preprocessed.rename(columns = {'ConferenceLocation_x':'ConferenceLocation'}, inplace=True)
df_mag_preprocessed = df_mag_preprocessed.drop(columns=['ConferenceLocation_y'])


if download_anyway:
    # Merge with the second location dataframe
    df_mag_preprocessed = pd.merge(df_mag_preprocessed, df_conf_locations_v1_2, on=['ConferenceNormalizedName'], how='left')

    # Combine the two columns
    df_mag_preprocessed['ConferenceLocation_x'] = df_mag_preprocessed['ConferenceLocation_x'].fillna(df_mag_preprocessed['ConferenceLocation_y'])
    df_mag_preprocessed.rename(columns = {'ConferenceLocation_x':'ConferenceLocation'}, inplace=True)
    df_mag_preprocessed = df_mag_preprocessed.drop(columns=['ConferenceLocation_y'])


# Merge with the third location dataframe
df_mag_preprocessed = pd.merge(df_mag_preprocessed, df_conf_locations_v2_1, on=['ConferenceNormalizedName'], how='left')

# Combine the two columns
df_mag_preprocessed['ConferenceLocation_x'] = df_mag_preprocessed['ConferenceLocation_x'].fillna(df_mag_preprocessed['ConferenceLocation_y'])
df_mag_preprocessed.rename(columns = {'ConferenceLocation_x':'ConferenceLocation'}, inplace=True)
df_mag_preprocessed = df_mag_preprocessed.drop(columns=['ConferenceLocation_y'])


# Merge with the fourth location dataframe
df_mag_preprocessed = pd.merge(df_mag_preprocessed, df_conf_locations_v2_2, on=['ConferenceNormalizedName'], how='left')

# Combine the two columns
df_mag_preprocessed['ConferenceLocation_x'] = df_mag_preprocessed['ConferenceLocation_x'].fillna(df_mag_preprocessed['ConferenceLocation_y'])
df_mag_preprocessed.rename(columns = {'ConferenceLocation_x':'ConferenceLocation'}, inplace=True)
df_mag_preprocessed = df_mag_preprocessed.drop(columns=['ConferenceLocation_y'])

df_mag_preprocessed.iloc[:5]

Count of how many paper's conference locations are still missing

In [ ]:
n_missing = len(df_mag_preprocessed.index) - len(df_mag_preprocessed.dropna(subset = ['ConferenceLocation']).index)
print(f"{n_missing} missing paper's conference locations")

## Write of the Final CSV on Disk

In [ ]:
# Write of the resulting CSV on Disk
df_mag_preprocessed.to_csv(path_file_export + 'out_mag_citations_and_locations.csv')
print(f'Successfully Exported the Preprocessed CSV to {path_file_export}out_mag_citations_and_locations.csv')

Check of the Exported CSV to be sure that everything went fine.

In [ ]:
# Check of the Exported CSV
df_mag_exported_csv = pd.read_csv(path_file_export + 'out_mag_citations_and_locations.csv', low_memory=False)
df_mag_exported_csv

Order by citations count descending to see the articles with the most citations

In [ ]:
# Order by citations count descending to see the articles with the most citations
df_mag_exported_csv = df_mag_exported_csv.sort_values(by='CitationCount', ascending=False)
df_mag_exported_csv.iloc[:5]